In [1]:
import random
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt

from tensorflow.keras.layers import Input, Lambda, Dense, Flatten,Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.vgg19 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
import numpy as np
import os
import cv2

In [2]:
epochs = 50
batch_size = 32
margin = 1  # Margin for constrastive loss.
IMAGE_SIZE = 256
CHANNELS = 3
#EPOCHS = 50

In [3]:
train_path="smalldata/train"
test_path="smalldata/test"
val_path="smalldata/val"

In [4]:
x_train=[]

for folder in os.listdir(train_path):

    sub_path=train_path+"/"+folder

    for img in os.listdir(sub_path):

        image_path=sub_path+"/"+img

        img_arr=cv2.imread(image_path)

        img_arr=cv2.resize(img_arr,(256,256))

        x_train.append(img_arr)

In [5]:
x_test=[]

for folder in os.listdir(test_path):

    sub_path=test_path+"/"+folder

    for img in os.listdir(sub_path):

        image_path=sub_path+"/"+img

        img_arr=cv2.imread(image_path)

        img_arr=cv2.resize(img_arr,(256,256))

        x_test.append(img_arr)

'x_test=[]\n\nfor folder in os.listdir(test_path):\n\n    sub_path=test_path+"/"+folder\n\n    for img in os.listdir(sub_path):\n\n        image_path=sub_path+"/"+img\n\n        img_arr=cv2.imread(image_path)\n\n        img_arr=cv2.resize(img_arr,(256,256))\n\n        x_test.append(img_arr)'

In [6]:
x_val=[]

for folder in os.listdir(val_path):

    sub_path=val_path+"/"+folder

    for img in os.listdir(sub_path):

        image_path=sub_path+"/"+img

        img_arr=cv2.imread(image_path)

        img_arr=cv2.resize(img_arr,(256,256))

        x_val.append(img_arr)

In [7]:
train_x=np.array(x_train)
#test_x=np.array(x_test)
val_x=np.array(x_val)

In [8]:
train_x=train_x/255.0
test_x=test_x/255.0
val_x=val_x/255.0

'train_x=train_x/255.0\ntest_x=test_x/255.0\nval_x=val_x/255.0'

In [9]:
train_datagen = ImageDataGenerator(rescale = 1./255)
test_datagen = ImageDataGenerator(rescale = 1./255)
val_datagen = ImageDataGenerator(rescale = 1./255)

In [10]:
training_set = train_datagen.flow_from_directory(train_path,
                                                 target_size = (256, 256),
                                                 batch_size = 32,
                                                 class_mode = 'sparse')
test_set = test_datagen.flow_from_directory(test_path,
                                            target_size = (256, 256),
                                            batch_size = 32,
                                            class_mode = 'sparse')
val_set = val_datagen.flow_from_directory(val_path,
                                            target_size = (256, 256),
                                            batch_size = 32,
                                            class_mode = 'sparse')


Found 500 images belonging to 5 classes.
Found 500 images belonging to 5 classes.
Found 500 images belonging to 5 classes.


In [11]:
train_y=training_set.classes
test_y=test_set.classes
val_y=val_set.classes

In [12]:
training_set.class_indices

{'Tomato___Early_blight': 0,
 'Tomato___Late_blight': 1,
 'Tomato___Leaf_Mold': 2,
 'Tomato___Tomato_mosaic_virus': 3,
 'Tomato___healthy': 4}

In [13]:
train_y.shape,test_y.shape,val_y.shape

((500,), (500,), (500,))

In [14]:
train_x= train_x.astype("uint8")
#test_x= test_x.astype("uint8")
val_x= val_x.astype("uint8")

In [15]:
#x_train, x_val = train_x[:3000], train_x[3000:]
#y_train, y_val = train_y[:3000], train_y[3000:]
#del train_x, train_y

In [16]:
def make_pairs(x, y):
    """Creates a tuple containing image pairs with corresponding label.

    Arguments:
        x: List containing images, each index in this list corresponds to one image.
        y: List containing labels, each label with datatype of `int`.

    Returns:
        Tuple containing two numpy arrays as (pairs_of_samples, labels),
        where pairs_of_samples' shape is (2len(x), 2,n_features_dims) and
        labels are a binary array of shape (2len(x)).
    """

    num_classes = max(y) + 1
    digit_indices = [np.where(y == i)[0] for i in range(num_classes)]

    pairs = []
    labels = []

    for idx1 in range(len(x)):
        # add a matching example
        x1 = x[idx1]
        label1 = y[idx1]
        idx2 = random.choice(digit_indices[label1])
        x2 = x[idx2]

        pairs += [[x1, x2]]
        labels += [1]

        # add a non-matching example
        label2 = random.randint(0, num_classes - 1)
        while label2 == label1:
            label2 = random.randint(0, num_classes - 1)

        idx2 = random.choice(digit_indices[label2])
        x2 = x[idx2]

        pairs += [[x1, x2]]
        labels += [0]

    return np.array(pairs), np.array(labels).astype("float32")




In [17]:
# make train pairs
pairs_train, labels_train = make_pairs(train_x, train_y)
del train_x,train_y

In [18]:
'''# make test pairs
pairs_test, labels_test = make_pairs(x_test, y_test)'''

'# make test pairs\npairs_test, labels_test = make_pairs(x_test, y_test)'

In [19]:
# make validation pairs
pairs_val, labels_val = make_pairs(val_x, val_y)
del val_x,val_y

In [20]:
x_train_1 = pairs_train[:, 0]  # x_train_1.shape is (60000, 28, 28)
x_train_2 = pairs_train[:, 1]
del pairs_train

In [21]:
x_val_1 = pairs_val[:, 0]  # x_val_1.shape = (60000, 28, 28)
x_val_2 = pairs_val[:, 1]
del pairs_val

In [22]:
def visualize(pairs, labels, to_show=6, num_col=3, predictions=None, test=False):
    """Creates a plot of pairs and labels, and prediction if it's test dataset.

    Arguments:
        pairs: Numpy Array, of pairs to visualize, having shape
               (Number of pairs, 2, 28, 28).
        to_show: Int, number of examples to visualize (default is 6)
                `to_show` must be an integral multiple of `num_col`.
                 Otherwise it will be trimmed if it is greater than num_col,
                 and incremented if if it is less then num_col.
        num_col: Int, number of images in one row - (default is 3)
                 For test and train respectively, it should not exceed 3 and 7.
        predictions: Numpy Array of predictions with shape (to_show, 1) -
                     (default is None)
                     Must be passed when test=True.
        test: Boolean telling whether the dataset being visualized is
              train dataset or test dataset - (default False).

    Returns:
        None.
    """
        # Define num_row
    # If to_show % num_col != 0
    #    trim to_show,
    #       to trim to_show limit num_row to the point where
    #       to_show % num_col == 0
    #
    # If to_show//num_col == 0
    #    then it means num_col is greater then to_show
    #    increment to_show
    #       to increment to_show set num_row to 1
    num_row = to_show // num_col if to_show // num_col != 0 else 1

    # `to_show` must be an integral multiple of `num_col`
    #  we found num_row and we have num_col
    #  to increment or decrement to_show
    #  to make it integral multiple of `num_col`
    #  simply set it equal to num_row * num_col
    to_show = num_row * num_col

    # Plot the images
    fig, axes = plt.subplots(num_row, num_col, figsize=(5, 5))
    for i in range(to_show):

        # If the number of rows is 1, the axes array is one-dimensional
        if num_row == 1:
            ax = axes[i % num_col]
        else:
            ax = axes[i // num_col, i % num_col]

        ax.imshow(tf.concat([pairs[i][0], pairs[i][1]], axis=1), cmap="gray")
        ax.set_axis_off()
        if test:
            ax.set_title("True: {} | Pred: {:.5f}".format(labels[i], predictions[i][0]))
        else:
            ax.set_title("Label: {}".format(labels[i]))
    if test:
        plt.tight_layout(rect=(0, 0, 1.9, 1.9), w_pad=0.0)
    else:
        plt.tight_layout(rect=(0, 0, 1.5, 1.5))
    plt.show()

In [23]:
#visualize(pairs_train[:-1], labels_train[:-1], to_show=4, num_col=4)

In [24]:
# Provided two tensors t1 and t2
# Euclidean distance = sqrt(sum(square(t1-t2)))
def euclidean_distance(vects):
    """Find the Euclidean distance between two vectors.

    Arguments:
        vects: List containing two tensors of same length.

    Returns:
        Tensor containing euclidean distance
        (as floating point value) between vectors.
    """

    x, y = vects
    sum_square = tf.math.reduce_sum(tf.math.square(x - y), axis=1, keepdims=True)
    return tf.math.sqrt(tf.math.maximum(sum_square, tf.keras.backend.epsilon()))

input = layers.Input((256, 256, 3))
x = tf.keras.layers.BatchNormalization()(input)
x = layers.Conv2D(4, (5, 5), activation="tanh")(x)
x = layers.AveragePooling2D(pool_size=(2, 2))(x)
x = layers.Conv2D(16, (5, 5), activation="tanh")(x)
x = layers.AveragePooling2D(pool_size=(2, 2))(x)
x = layers.Flatten()(x)

x = tf.keras.layers.BatchNormalization()(x)
x = layers.Dense(10, activation="tanh")(x)
embedding_network = keras.Model(input, x)


input_1 = layers.Input((256, 256, 3))
input_2 = layers.Input((256, 256, 3))

# As mentioned above, Siamese Network share weights between
# tower networks (sister networks). To allow this, we will use
# same embedding network for both tower networks.
tower_1 = embedding_network(input_1)
tower_2 = embedding_network(input_2)

merge_layer = layers.Lambda(euclidean_distance)([tower_1, tower_2])
normal_layer = tf.keras.layers.BatchNormalization()(merge_layer)
output_layer = layers.Dense(1, activation="sigmoid")(normal_layer)
siamese = keras.Model(inputs=[input_1, input_2], outputs=output_layer)

In [25]:
def loss(margin=1):
    """Provides 'constrastive_loss' an enclosing scope with variable 'margin'.

  Arguments:
      margin: Integer, defines the baseline for distance for which pairs
              should be classified as dissimilar. - (default is 1).

  Returns:
      'constrastive_loss' function with data ('margin') attached.
  """

    # Contrastive loss = mean( (1-true_value) * square(prediction) +
    #                         true_value * square( max(margin-prediction, 0) ))
    def contrastive_loss(y_true, y_pred):
        """Calculates the constrastive loss.

      Arguments:
          y_true: List of labels, each label is of type float32.
          y_pred: List of predictions of same length as of y_true,
                  each label is of type float32.

      Returns:
          A tensor containing constrastive loss as floating point value.
      """

        square_pred = tf.math.square(y_pred)
        margin_square = tf.math.square(tf.math.maximum(margin - (y_pred), 0))
        return tf.math.reduce_mean(
            (1 - y_true) * square_pred + (y_true) * margin_square
        )

    return contrastive_loss

In [26]:
siamese.compile(loss=loss(margin=margin), optimizer="RMSprop", metrics=["accuracy"])
siamese.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 input_3 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 model (Functional)             (None, 10)           835446      ['input_2[0][0]',                
                                                                  'input_3[0][0]']          

In [27]:
'''gpus = tf.config.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
  try:
    tf.config.set_visible_devices(gpus[0], 'GPU')
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
  except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
    print(e)'''

'gpus = tf.config.list_physical_devices(\'GPU\')\nif gpus:\n  # Restrict TensorFlow to only use the first GPU\n  try:\n    tf.config.set_visible_devices(gpus[0], \'GPU\')\n    logical_gpus = tf.config.list_logical_devices(\'GPU\')\n    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")\n  except RuntimeError as e:\n    # Visible devices must be set before GPUs have been initialized\n    print(e)'

In [28]:
history = siamese.fit(
    [x_train_1, x_train_2],
    labels_train,
    validation_data=([x_val_1, x_val_2], labels_val),
    batch_size=batch_size,
    epochs=epochs,
)

Epoch 1/50
32/32 [==============================] - 5s 74ms/step - loss: 0.2610 - accuracy: 0.4610 - val_loss: 0.2564 - val_accuracy: 0.4960
Epoch 2/50
32/32 [==============================] - 2s 58ms/step - loss: 0.2568 - accuracy: 0.4660 - val_loss: 0.2616 - val_accuracy: 0.4650
Epoch 3/50
32/32 [==============================] - 2s 57ms/step - loss: 0.2567 - accuracy: 0.4690 - val_loss: 0.2573 - val_accuracy: 0.4570
Epoch 4/50
32/32 [==============================] - 2s 58ms/step - loss: 0.2561 - accuracy: 0.4760 - val_loss: 0.2564 - val_accuracy: 0.4780
Epoch 5/50
32/32 [==============================] - 2s 59ms/step - loss: 0.2556 - accuracy: 0.4730 - val_loss: 0.2545 - val_accuracy: 0.4810
Epoch 6/50
32/32 [==============================] - 2s 58ms/step - loss: 0.2525 - accuracy: 0.4950 - val_loss: 0.2543 - val_accuracy: 0.4780
Epoch 7/50
32/32 [==============================] - 2s 58ms/step - loss: 0.2512 - accuracy: 0.4970 - val_loss: 0.2534 - val_accuracy: 0.5010
Epoch 8/50
32

In [1]:
new_model= load_model("model.h5")

NameError: name 'load_model' is not defined